### Initial package imports

In [3]:
import pandas as pd
import math


## Initial files being read in and dataset creation

In [7]:
# Read the data
main_df = pd.read_csv('Data/Cleaned_Indices_Assignment1.csv', sep=';')

# Read the interest rate data
#interest_rate_df = pd.read_csv('Data/ECB_Rates_2012_to_2022.csv', sep=';')
interest_rate_bond_df = pd.read_csv('Data/ECB_Data_10yr_Treasury_bond.csv', sep=',')
# Divide all values in the interest_rate_bond_df (except the Date column) by a certain number, e.g., 100, and store it in another variable
scaled_interest_rate_bond_df = interest_rate_bond_df.copy()
scaled_interest_rate_bond_df.iloc[:, 1:] = scaled_interest_rate_bond_df.iloc[:, 1:] / 100
print(scaled_interest_rate_bond_df.head())

# Convert date columns to datetime format for proper merging
main_df['Date'] = pd.to_datetime(main_df['Date'], format='%d-%m-%Y')
#interest_rate_df['Date'] = pd.to_datetime(interest_rate_df['Date'], format='%d-%m-%Y')
#gov_bond_investment_df['Date'] = pd.to_datetime(gov_bond_investment_df['Date'], format='%Y-%m-%d')
interest_rate_bond_df['Date'] = pd.to_datetime(interest_rate_bond_df['Date'], format='%Y-%m-%d')

# Merge the dataframes on the Date column
main_df = pd.merge(main_df, interest_rate_bond_df, on='Date', how='left')
#main_df = pd.merge(main_df, gov_bond_investment_df, on='Date', how='left')


TypeError: unsupported operand type(s) for /: 'str' and 'int'

## Portfolio details

### Instruments:
- **S&P500**
- **DAX40**
- **NIKKEI**
- **EU Government Bond (10-year maturity, AAA-rated)**

### Invested amount:
- **10,000,000 EURO**

### Period:
- **01/01/2012 - 31/12/2022**

### Weights:
- **S&P500**: 0.4  
- **DAX40**: 0.3  
- **NIKKEI**: 0.15  
- **EU Government Bond**: 0.15  

### Measures:
- **Value at Risk (VaR)**: 1, 5, 10 days  
- **Expected Shortfall (ES)**  



In [12]:
main_df


,Date,S&P500_Closing,Dax40_Closing,Nikkei_Closing,U_S&P500_Returns,U_Dax40_Returns,U_Nikkei_Returns,U_S&P500_Loss,U_Dax40_Loss,U_Nikkei_Loss,C_S&P500_Returns,C_Dax40_Returns,C_Nikkei_Returns,C_S&P500_Loss,C_Dax40_Loss,C_Nikkei_Loss,TIME PERIOD,"Yield curve spot rate, 10-year maturity - Government bond, nominal, all issuers whose rating is triple A - Euro area (changing composition) (YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_10Y)"
0,2012-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-01-02,NaN,"6.075,52",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02 Jan 2012,2.717862
2,2012-01-03,"1.277,06","6.166,57",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03 Jan 2012,2.744522
3,2012-01-04,"1.277,30","6.111,55","8.560,11",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04 Jan 2012,2.776691
4,2012-01-05,"1.281,06","6.095,99","8.488,71","0,002939385","-0,002549246","-0,008375995","-3,76","15,56","71,4","0,002939385","-0,003257936","-0,000108385","-3,76","19,88568","91,2492",05 Jan 2012,2.784807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2866,2022-12-26,"3.844,82","13.940,93","26.405,87",0,0,"0,006482408",0,0,"-170,62",0,0,"4,87801E-05",0,0,"-181,45437",NaN,NaN
2867,2022-12-27,"3.829,25","13.995,10","26.447,87","-0,004057826","0,003878151","0,001589292","15,57","-54,17",-42,"-0,004057826","0,004125577","1,19054E-05","15,57","-57,626046","-44,6796",27 Dec 2022,2.501054
2868,2022-12-28,"3.783,22","13.925,60","26.340,50","-0,012093463","-0,004978395","-0,004067947","46,03","69,5","107,37","-0,012093463","-0,005281082","-3,02493E-05","46,03","73,7256","113,898096",28 Dec 2022,2.522043
2869,2022-12-29,"3.849,28","14.071,72","26.093,67","0,017310619","0,010438237","-0,009414922","-66,06","-146,12","246,83","0,017310619","0,011128204","-7,07814E-05","-66,06","-155,778532","263,145463",29 Dec 2022,2.526705
